In [1]:
import sys
sys.path.append('src')
import numpy as np
import tensorflow as tf
from bridges import Model

In [ ]:
m = Model()

In [3]:
m.gen('blah blah')

['blah blahustom to put hand on']

In [6]:
m.get_perplexity(['blah blah', 'blah blah les loups vont au bois'])

(Batch size changed from 1 to 2, resetting graph.)


array([6.7786068e-08, 2.7807615e+01], dtype=float32)

In [8]:
for s in ['blah blah', 'blah blah les loups vont au bois']:
    print(s, m.get_perplexity(s))

blah blah [0.47583383]
blah blah les loups vont au bois [27.80754]


In [29]:
def print_lists(ll):
    if not isinstance(ll[0],(list, tuple, np.ndarray)):
        print(ll)
    else:
        print(*ll, sep='\n')    

In [30]:
def normalize(logits, verbose=False):
    mu = np.mean(logits, axis=-1, keepdims=True)
    lm = logits - mu
    le = np.exp(lm)
    logprobs = le / np.sum(le, axis=-1, keepdims=True)
    if verbose:
        return logprobs, mu, lm, le
    return logprobs

In [37]:
logprobs, mu, lm, le = normalize(logits, verbose=True)

In [38]:
np.mean(logits[0,0])

7.638048

In [39]:
mu[0,0]

array([7.638048], dtype=float32)

In [40]:
lm[0,0,0] == logits[0,0,0] - mu[0,0]

array([ True])

In [41]:
np.exp(lm[0,0,0]) == le[0,0,0]

True

In [42]:
np.sum(le, axis=-1, keepdims=True).shape

(4, 12, 1)

In [43]:
np.sum(le, axis=-1, keepdims=True)[0,0]

array([14673772.], dtype=float32)

In [44]:
logprobs[0,0,0] == le[0,0,0] / np.sum(le, axis=-1, keepdims=True)[0,0, 0]

True

In [207]:
def get_perplexity(model, sentences=["\n"], batch_size=None, verbose=False):
#     # save current batch_size
#     batch_size_bak=None
#     if batch_size is not None and self.batch_size != batch_size:
#         batch_size_bak = self.batch_size
#     model.check_batch_size(batch_size)
    if isinstance(sentences, str):
        sentences = [sentences]
    tkns = model.pad_sequences(sentences)
    print('sequences:')
    print_lists(tkns)
    
    # don't take the last one (predicting the token after our sentence)        
    logits = model.get_logits(context_tokens=tkns)[:, :-1, :] 
    
    print("logits, shape:", logits.shape)
    
    # normalization between 0 and 1
    # most importantly: make all 
    logprobs = normalize(logits)
    print("logprobs, shape:", logprobs.shape)
 
    scores = np.nan_to_num([
        [logprobs[j, i, token] for i, token in enumerate(seq[:-1])] for j, seq in enumerate(tkns)
    ])    
    print("scores, shape:", scores.shape)
    perplexities = 2 ** (-np.mean(np.log2(scores), axis=-1))
    return perplexities

#     # restoring batch size 
#     if batch_size_bak: model.check_batch_size(batch_size_bak)    
#     return perplexities

In [208]:
snt = [
    "Ah, nous voilà bien",
    "Je me baladais dans le gris sans fond",
    "LE COMTE. Quel déshonneur !",
    "Ah."
]

In [211]:
m.get_logits(context_tokens=m.pad_sequences(snt)).shape

(4, 12, 50257)

In [212]:
m.get_logits(context_tokens=m.pad_sequences(snt))[:, :-1, :].shape

(4, 11, 50257)

In [209]:
print(*zip(snt, get_perplexity(m, snt)), sep='\n')

sequences:
[  220   220   220 10910    11   299   516  7608   346 24247   275  2013]
[40932   502  3652  4763   271   288   504   443  1036   271 38078 16245]
[  220  2538  9440  9328    13  2264   417 39073  1477 47476   333  5145]
[  220   220   220   220   220   220   220   220   220   220 10910    13]
logits, shape: (4, 11, 50257)
logprobs, shape: (4, 11, 50257)
scores, shape: (4, 11)
('Ah, nous voilà bien', 77367.34)
('Je me baladais dans le gris sans fond', 188906.8)
('LE COMTE. Quel déshonneur !', 2907181.0)
('Ah.', 2.7888317)


In [48]:
padded = m.pad_sequences(snt)
logits = m.get_logits(padded)

In [49]:
logits.shape

(4, 12, 50257)

In [340]:
logprobs = normalize(logits)

In [341]:
logprobs.shape

(4, 12, 50257)

In [343]:
scores = np.nan_to_num([[logprobs[j, i, token] 
 for i, token in enumerate(batch[:-1])]
 for j, batch in enumerate(padded)])

In [344]:
perplexities = 2 ** (-np.mean(np.log2(scores), axis=-1))

In [347]:
print(*perplexities, sep='\n')

77367.34
188906.8
2907181.0
2.7888317


In [251]:
logits[0]

array([[14.289879 ,  9.88566  , 10.730696 , ...,  5.7731833,  6.458465 ,
         5.841981 ],
       [ 3.1374109,  1.3310282,  4.4881535, ...,  3.720184 ,  3.8290942,
         3.7567344],
       [ 3.4757946,  0.8460077,  4.568767 , ...,  3.2985313,  3.3040564,
         3.3131304],
       ...,
       [ 2.0084343, -8.294467 , -6.12812  , ..., -9.5075   , -9.178121 ,
        -9.197476 ],
       [ 7.3087406,  6.4965568,  6.595199 , ...,  1.4063548,  2.3194256,
         1.880352 ],
       [ 3.4282572, -8.777307 , -6.4584146, ..., -9.818342 , -9.568864 ,
        -9.863892 ]], dtype=float32)

In [56]:
%timeit [(i, tkn) for i, tkn in enumerate(padded[0])]

2.43 µs ± 9.62 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [58]:
[(i, tkn) for i, tkn in enumerate(padded[0])]

[(0, 220),
 (1, 220),
 (2, 220),
 (3, 10910),
 (4, 11),
 (5, 299),
 (6, 516),
 (7, 7608),
 (8, 346),
 (9, 24247),
 (10, 275),
 (11, 2013)]

In [68]:
padded[0].size

12

In [83]:
padded[0]

array([  220,   220,   220, 10910,    11,   299,   516,  7608,   346,
       24247,   275,  2013], dtype=int32)

In [86]:
%timeit list(zip(np.arange(padded[0].size), padded[0]))

4.02 µs ± 26.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [97]:
%timeit np.array(zip(range(padded[0].size), padded[0]))

2.41 µs ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [96]:
%timeit list(zip(range(padded[0].size), padded[0]))

2.07 µs ± 21.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [103]:
%timeit list(zip(range(padded.shape[1]), padded[0]))

1.82 µs ± 25.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [170]:
padded[0]

array([  220,   220,   220, 10910,    11,   299,   516,  7608,   346,
       24247,   275,  2013], dtype=int32)

In [182]:
np.broadcast_to(np.arange(padded.shape[0]), (padded.shape[1],padded.shape[0])).T

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]])

In [178]:
np.broadcast_to(np.arange(padded.shape[1]), padded.shape)

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]])

In [183]:
np.stack([
    np.broadcast_to(np.arange(padded.shape[0]), (padded.shape[1],padded.shape[0])).T,
    np.broadcast_to(np.arange(padded.shape[1]), padded.shape), 
    padded
], axis=-1)

array([[[    0,     0,   220],
        [    0,     1,   220],
        [    0,     2,   220],
        [    0,     3, 10910],
        [    0,     4,    11],
        [    0,     5,   299],
        [    0,     6,   516],
        [    0,     7,  7608],
        [    0,     8,   346],
        [    0,     9, 24247],
        [    0,    10,   275],
        [    0,    11,  2013]],

       [[    1,     0, 40932],
        [    1,     1,   502],
        [    1,     2,  3652],
        [    1,     3,  4763],
        [    1,     4,   271],
        [    1,     5,   288],
        [    1,     6,   504],
        [    1,     7,   443],
        [    1,     8,  1036],
        [    1,     9,   271],
        [    1,    10, 38078],
        [    1,    11, 16245]],

       [[    2,     0,   220],
        [    2,     1,  2538],
        [    2,     2,  9440],
        [    2,     3,  9328],
        [    2,     4,    13],
        [    2,     5,  2264],
        [    2,     6,   417],
        [    2,     7, 39073],
    

In [184]:
indices = np.stack([
            np.broadcast_to(np.arange(padded.shape[0]), (padded.shape[1],padded.shape[0])).T,
            np.broadcast_to(np.arange(padded.shape[1]), padded.shape), 
            padded
        ], axis=-1)

In [202]:
indices

array([[[    0,     0,   220],
        [    0,     1,   220],
        [    0,     2,   220],
        [    0,     3, 10910],
        [    0,     4,    11],
        [    0,     5,   299],
        [    0,     6,   516],
        [    0,     7,  7608],
        [    0,     8,   346],
        [    0,     9, 24247],
        [    0,    10,   275],
        [    0,    11,  2013]],

       [[    1,     0, 40932],
        [    1,     1,   502],
        [    1,     2,  3652],
        [    1,     3,  4763],
        [    1,     4,   271],
        [    1,     5,   288],
        [    1,     6,   504],
        [    1,     7,   443],
        [    1,     8,  1036],
        [    1,     9,   271],
        [    1,    10, 38078],
        [    1,    11, 16245]],

       [[    2,     0,   220],
        [    2,     1,  2538],
        [    2,     2,  9440],
        [    2,     3,  9328],
        [    2,     4,    13],
        [    2,     5,  2264],
        [    2,     6,   417],
        [    2,     7, 39073],
    

In [204]:
logits.shape

(4, 12, 50257)

In [176]:
np.stack([np.broadcast_to(np.arange(padded.shape[1]), padded.shape), padded], axis=-1).shape

(4, 12, 2)

In [175]:
np.stack([np.broadcast_to(np.arange(padded.shape[1]), padded.shape), padded], axis=-1)

array([[[    0,   220],
        [    1,   220],
        [    2,   220],
        [    3, 10910],
        [    4,    11],
        [    5,   299],
        [    6,   516],
        [    7,  7608],
        [    8,   346],
        [    9, 24247],
        [   10,   275],
        [   11,  2013]],

       [[    0, 40932],
        [    1,   502],
        [    2,  3652],
        [    3,  4763],
        [    4,   271],
        [    5,   288],
        [    6,   504],
        [    7,   443],
        [    8,  1036],
        [    9,   271],
        [   10, 38078],
        [   11, 16245]],

       [[    0,   220],
        [    1,  2538],
        [    2,  9440],
        [    3,  9328],
        [    4,    13],
        [    5,  2264],
        [    6,   417],
        [    7, 39073],
        [    8,  1477],
        [    9, 47476],
        [   10,   333],
        [   11,  5145]],

       [[    0,   220],
        [    1,   220],
        [    2,   220],
        [    3,   220],
        [    4,   220],
        [ 

In [135]:
np.stack(padded.shape[0]*[np.arange(padded.shape[1]), padded], axis=-1)

ValueError: all input arrays must have the same shape

In [119]:
[list(zip(range(padded.shape[1]), padded[b])) for b in range(padded.shape[0])]

[[(0, 220),
  (1, 220),
  (2, 220),
  (3, 10910),
  (4, 11),
  (5, 299),
  (6, 516),
  (7, 7608),
  (8, 346),
  (9, 24247),
  (10, 275),
  (11, 2013)],
 [(0, 40932),
  (1, 502),
  (2, 3652),
  (3, 4763),
  (4, 271),
  (5, 288),
  (6, 504),
  (7, 443),
  (8, 1036),
  (9, 271),
  (10, 38078),
  (11, 16245)],
 [(0, 220),
  (1, 2538),
  (2, 9440),
  (3, 9328),
  (4, 13),
  (5, 2264),
  (6, 417),
  (7, 39073),
  (8, 1477),
  (9, 47476),
  (10, 333),
  (11, 5145)],
 [(0, 220),
  (1, 220),
  (2, 220),
  (3, 220),
  (4, 220),
  (5, 220),
  (6, 220),
  (7, 220),
  (8, 220),
  (9, 220),
  (10, 10910),
  (11, 13)]]

In [106]:
list(zip(range(padded.shape[0]), list(zip(range(padded.shape[1]), padded[0]))))

[(0, (0, 220)), (1, (1, 220)), (2, (2, 220)), (3, (3, 10910))]

In [247]:
padded[0,:,None]

array([[  220],
       [  220],
       [  220],
       [10910],
       [   11],
       [  299],
       [  516],
       [ 7608],
       [  346],
       [24247],
       [  275],
       [ 2013]], dtype=int32)

In [ ]:
indices = [(b, i, tkn) for  in ]

In [167]:
list(zip(np.arange(len(padded[0])),padded[0]))

[(0, 220),
 (1, 220),
 (2, 220),
 (3, 10910),
 (4, 11),
 (5, 299),
 (6, 516),
 (7, 7608),
 (8, 346),
 (9, 24247),
 (10, 275),
 (11, 2013)]

In [238]:
for j, (b, tkns) in enumerate(zip(logits, padded)):
    for i, tkn in enumerate(tkns):
        print(f"{b[i,tkn]:>6.2f}", end=' ')
    print()

 22.68  18.83  18.52  -0.17  -7.89  -0.47  -9.74  -3.58   3.29  -7.50   0.54  -6.90 
  5.66   4.03   4.06 -11.92 -11.42   0.89  -4.92  -2.19   0.91  -5.07  -4.36  -2.13 
 22.68   1.53   7.14   8.36   0.53   1.24  -1.31  -0.13   0.08 -10.10  -5.05   2.72 
 22.68  18.83  18.52  18.08  19.06  20.29  21.18  21.54  21.77  21.78   1.73   0.93 


In [236]:
for j, (b, tkns) in enumerate(zip(logits, padded)):
    print('batch:', j)
    print('tokens:\t', end='')
    for tkn in tkns:
        print(f"{tkn:>6} | ", end='')
    print()
    print('logits:\t', end='')
    for i, tkn in enumerate(tkns):
        print(f"{b[i,tkn]:>6.2f} |", end=' ')
    print()

batch: 0
tokens:	   220 |    220 |    220 |  10910 |     11 |    299 |    516 |   7608 |    346 |  24247 |    275 |   2013 | 
logits:	 22.68 |  18.83 |  18.52 |  -0.17 |  -7.89 |  -0.47 |  -9.74 |  -3.58 |   3.29 |  -7.50 |   0.54 |  -6.90 | 
batch: 1
tokens:	 40932 |    502 |   3652 |   4763 |    271 |    288 |    504 |    443 |   1036 |    271 |  38078 |  16245 | 
logits:	  5.66 |   4.03 |   4.06 | -11.92 | -11.42 |   0.89 |  -4.92 |  -2.19 |   0.91 |  -5.07 |  -4.36 |  -2.13 | 
batch: 2
tokens:	   220 |   2538 |   9440 |   9328 |     13 |   2264 |    417 |  39073 |   1477 |  47476 |    333 |   5145 | 
logits:	 22.68 |   1.53 |   7.14 |   8.36 |   0.53 |   1.24 |  -1.31 |  -0.13 |   0.08 | -10.10 |  -5.05 |   2.72 | 
batch: 3
tokens:	   220 |    220 |    220 |    220 |    220 |    220 |    220 |    220 |    220 |    220 |  10910 |     13 | 
logits:	 22.68 |  18.83 |  18.52 |  18.08 |  19.06 |  20.29 |  21.18 |  21.54 |  21.77 |  21.78 |   1.73 |   0.93 | 


In [53]:
for i, a in enumerate(padded[:, :-1]):
    print(i, a)

0 [  220   220   220 10910    11   299   516  7608   346 24247   275]
1 [40932   502  3652  4763   271   288   504   443  1036   271 38078]
2 [  220  2538  9440  9328    13  2264   417 39073  1477 47476   333]
3 [  220   220   220   220   220   220   220   220   220   220 10910]


---

## Logits

In [34]:
def get_logits(model, context_tokens, last_only=False):
    """
    Generate the logits (probabilities of each token) at each step for a
    given one or more sequences of tokens.
    Returns:
    --------
        logits: array of shape: (batch_size, n_tokens, n_vocab)
                or, if last_only is True: (batch_size, 1, n_vocab)
    """
    # model.check_batch_size(batch_size)
    if not isinstance(context_tokens[0], (list, tuple, np.ndarray)):
        model.check_batch_size(1)
        context = model.batch_size * [context_tokens]
        print('one dim')
        print(context)
    else:
        model.check_batch_size(len(context_tokens))
        context = context_tokens
        print('many dims')        
        print(context)
    # model.model, defined at initialization, returns logits & attention matrix
    logits = model.sess.run(model.model, feed_dict={model.context: context})["logits"]
    # all logits starting from the second token, n logits for n tokens
    # shape (batch_size, n_tokens, vocab_size)
    if not last_only:
        return logits
    # logits for next token, None to keep dims intact
    return logits[:, None, -1, :]

In [35]:
get_logits(m, m.encode(snt[0])).shape

one dim
[array([10910,    11,   299,   516,  7608,   346, 24247,   275,  2013])]


(1, 9, 50257)

In [27]:
get_logits(m, m.pad_sequences(snt)).shape

many dims
[[  220   220   220 10910    11   299   516  7608   346 24247   275  2013]
 [40932   502  3652  4763   271   288   504   443  1036   271 38078 16245]
 [  220  2538  9440  9328    13  2264   417 39073  1477 47476   333  5145]
 [  220   220   220   220   220   220   220   220   220   220 10910    13]]


(4, 12, 50257)

In [71]:
get_logits(m, snt)[:, :-1, :].shape

(4, 11, 50257)

In [64]:
get_logits(m, snt, last_only=True).shape

(4, 1, 50257)

---

## Sequence padding

In [9]:
m.pad_sequences(['blah blah', 'blah diblah'])

array([[  220,   220,  2436,   993, 33367],
       [ 2436,   993,   288,   571,  9271]], dtype=int32)

In [16]:
tf.keras.preprocessing.sequence.pad_sequences(
    m.encode(snt), 
    maxlen=None, 
    dtype='int32', 
    padding='pre', 
    truncating='pre',
    value=220
)

array([[  220,   220,   220, 10910,    11,   299,   516,  7608,   346,
        24247,   275,  2013],
       [40932,   502,  3652,  4763,   271,   288,   504,   443,  1036,
          271, 38078, 16245],
       [  220,  2538,  9440,  9328,    13,  2264,   417, 39073,  1477,
        47476,   333,  5145],
       [  220,   220,   220,   220,   220,   220,   220,   220,   220,
          220, 10910,    13]], dtype=int32)

In [20]:
m.encode(' ')

array([220])

After having coded the function into the class (220 is space):

In [19]:
print(m.pad_sequences(snt, padding='pre'))
m.decode(m.pad_sequences(snt, padding='pre'))

[[  220   220   220 10910    11   299   516  7608   346 24247   275  2013]
 [40932   502  3652  4763   271   288   504   443  1036   271 38078 16245]
 [  220  2538  9440  9328    13  2264   417 39073  1477 47476   333  5145]
 [  220   220   220   220   220   220   220   220   220   220 10910    13]]


['   Ah, nous voilà bien',
 'Je me baladais dans le gris sans fond',
 ' LE COMTE. Quel déshonneur !',
 '          Ah.']

In [18]:
print(m.pad_sequences(snt, padding='post'))
m.decode(m.pad_sequences(snt, padding='post'))

[[10910    11   299   516  7608   346 24247   275  2013   220   220   220]
 [40932   502  3652  4763   271   288   504   443  1036   271 38078 16245]
 [ 2538  9440  9328    13  2264   417 39073  1477 47476   333  5145   220]
 [10910    13   220   220   220   220   220   220   220   220   220   220]]


['Ah, nous voilà bien   ',
 'Je me baladais dans le gris sans fond',
 'LE COMTE. Quel déshonneur ! ',
 'Ah.          ']

Encoding and decoding in turn seem to yield identity.

In [20]:
print(m.encode(m.decode(m.pad_sequences(snt, padding='post'))) == m.pad_sequences(snt, padding='post'))

[[ True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True]]
